In [1]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

backup_write("src/careeragent/api/run_manager_service.py", r'''
from __future__ import annotations

import asyncio
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import (
    run_single_layer,
    approve_ranking_flow,
    approve_drafts_flow,
)

try:
    from careeragent.config import artifacts_root  # type: ignore
except Exception:
    def artifacts_root() -> Path:
        return Path("src/careeragent/artifacts").resolve()


def utc_now() -> str:
    """
    Description: Return UTC timestamp string.
    Layer: L0
    Input: None
    Output: ISO UTC string
    """
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


class RunManagerService:
    """
    Description: Background run manager that persists state after each layer for real-time UI.
    Layer: L8
    Input: resume + preferences + actions
    Output: persisted state snapshots for polling
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()
        self._lock = threading.Lock()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        """
        Description: Create new run record and persist initial state.
        Layer: L8
        Input: resume + prefs
        Output: created state snapshot
        """
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)

        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            default_th = float(thresholds["default"])
            thresholds.setdefault("parser", default_th)
            thresholds.setdefault("discovery", default_th)
            thresholds.setdefault("match", default_th)
            thresholds.setdefault("draft", default_th)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scraped": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {
                "created_at_utc": utc_now(),
                "heartbeat_utc": utc_now(),
                "last_layer": None,
            },
            "steps": [],  # ✅ progress-friendly timeline
            "live_feed": [{"layer": "L1", "agent": "API", "message": "Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        """
        Description: Load run state snapshot.
        Layer: L8
        """
        return self._store.get_state(run_id=run_id)

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        """
        Description: Persist run state snapshot.
        Layer: L8
        """
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(
            run_id=run_id,
            status=str(state.get("status", "unknown")),
            state=state,
            updated_at_utc=utc_now(),
        )

    def start_background(self, run_id: str) -> None:
        """
        Description: Start L0→L5 pipeline in background thread.
        Layer: L6
        """
        t = threading.Thread(target=self._bg_runner, args=(run_id,), daemon=True)
        t.start()

    def _bg_runner(self, run_id: str) -> None:
        asyncio.run(self._bg_async(run_id))

    async def _run_layer_with_timeout(self, state: Dict[str, Any], layer: str, timeout_sec: int) -> Dict[str, Any]:
        """
        Description: Run a single layer with timeout protection.
        Layer: L6
        """
        try:
            return await asyncio.wait_for(run_single_layer(state, layer), timeout=timeout_sec)
        except asyncio.TimeoutError:
            state["status"] = "needs_human_approval"
            state["pending_action"] = f"timeout_{layer.lower()}"
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "TimeoutGuard", "message": f"{layer} timed out after {timeout_sec}s."})
            return state

    async def _bg_async(self, run_id: str) -> None:
        """
        Description: Stream state updates after each layer so UI progress moves in real-time.
        Layer: L6
        """
        state = self.get_state(run_id)
        if not state:
            return

        # run L0 → L5 and save after each layer
        plan = [
            ("L0", 45),
            ("L2", 60),
            ("L3", 120),
            ("L4", 180),
            ("L5", 45),
        ]

        for layer, tmo in plan:
            # stop if already blocked/hitl
            if state.get("status") in ("blocked", "needs_human_approval", "failed", "completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer
            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent": "Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            state = await self._run_layer_with_timeout(state, layer, tmo)

            # mark step done
            state["steps"][-1]["status"] = state.get("status", "running")
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            # if ranking is ready, we intentionally stop for HITL
            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        # If reached here, ensure HITL at ranking (expected flow)
        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"Orchestrator","message":"Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def execute_layer(self, *, run_id: str, layer: str) -> Dict[str, Any]:
        """
        Description: Engineer view — execute a single layer node.
        Layer: L6
        """
        state = self.get_state(run_id) or {}
        state = await run_single_layer(state, layer.upper())
        self.save_state(run_id=run_id, state=state)
        return state

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Description: HITL & operational actions.
        Layer: L5
        """
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        state.setdefault("meta", {})
        state["meta"]["last_user_action"] = {"type": action_type, "payload": payload, "at_utc": utc_now()}

        if action_type == "execute_layer":
            layer = str(payload.get("layer", "")).upper()
            state = await run_single_layer(state, layer)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_ranking":
            state = await approve_ranking_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_drafts":
            state = await approve_drafts_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        # keep your existing handlers (resume_cleanup_submit, reject_ranking, etc.) in your file if you already added them.
        # For safety: do not break unknown actions.
        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''')

print("✅ Patched RunManagerService to persist state after each layer (real-time progress).")
print("Restart backend + UI now.")

✅ CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/api/run_manager_service.py
✅ Patched RunManagerService to persist state after each layer (real-time progress).
Restart backend + UI now.


In [ ]:
RUN_ID=42d3b95ff4a94fe68228c0ff08b1726c
curl -sS -i "http://127.0.0.1:8000/status/$RUN_ID" | head -n 40